In [57]:
# Import

import pandas as pd
import http.client
import matplotlib 
import json
import math
import statistics
import matplotlib.pyplot as plt

In [58]:
# Settings

type_int = 5*60
type = "5min"
symbol = "BTC-USDT"
fromDate = "2000.3.1"
toDate = ""
pages = 1
window = 200


Indicators=True
Candle_patterns=False


In [59]:
# Connection

conn = http.client.HTTPSConnection("api.kucoin.com")
if toDate != "":
    EndAt = int(pd.Timestamp(toDate).timestamp())
else:
    conn.request("GET", "/api/v1/timestamp")
    STime = conn.getresponse()
    dataServerTime = json.loads(STime.read())['data'] / 1000
    EndAt = int(dataServerTime)
totalData = []
for page in range(0, pages):
    StartAt = str(int(pd.Timestamp(fromDate).timestamp()))
    EndAt = EndAt-(page*type_int*1500)
    conn.request("GET", f"/api/v1/market/candles?type={type}&symbol={symbol}&startAt={StartAt}&endAt={EndAt}")
    res = conn.getresponse()
    data = res.read()
    for candle in json.loads(data)['data']:
        totalData.append(candle)


In [60]:
# Total data frame

df = pd.DataFrame(totalData)
df = df.rename(columns={0: 'Time', 1: 'Open', 2: 'Close', 3: 'High', 4: 'Low', 5: 'Volume', 6: 'Transaction'})
#df['Time'] = pd.to_datetime(df['Time'], unit='s')
# df.set_index('Time', inplace = True)
df[['Open', 'Close', 'High', 'Low']] = df[['Open', 'Close', 'High', 'Low']].astype(float)
df[['Volume', 'Transaction']] = df[['Volume', 'Transaction']].astype(float)
df['Average'] = (df['Open'] + df['Close']) / 2
window_df= df[0:window]

In [ ]:
# Indicators

if Indicators == True:
    # Simple Moving Average(SMA):
    class Indicator_SMA():
        def main(self, df: pd.DataFrame, period: int) -> pd.DataFrame:
            SMA = []

            for i in range(0, len(df)-period):
                SMA.append((df['Close'][i:i+period]).mean())
            for n in range(0, period):
                SMA.append(0)

            df[f'SMA_{period}'] = SMA
            return df

        def eval(self, df: pd.DataFrame, period: int):
            df = self.main(df, period)
            plt.figure(figsize=(15, 5), dpi=300)
            plt.plot(df.index, df.SMA_12, label="SMA")
            plt.plot(df.index, df.Close, label="Price")
            plt.xlim(df.index[200], 0)
            plt.ylim(bottom=42000, top=44000)
            plt.grid()
            plt.legend()
            plt.show()

    # Exponential Moving Average(EMA):

    class Indicator_EMA():
        def main(self, df: pd.DataFrame, period: int):
            EMA = []

            k = 2/(period+1)

            for i in range(0, period):
                EMA.append(0)

            EMA.append((df['Close'][(len(df)-period):len(df)]).mean())

            for i in range(period, len(df)-1):
                EMA.append(
                    ((df['Close'][(len(df)-1)-i])*k) + ((EMA[-1])*(1-k)))

            EMA.reverse()
            df[f'EMA_{period}'] = EMA
            return df

        def eval(self, df: pd.DataFrame):
            pass

    # Moving Average Convergence/Divergence(MACD):

    class Indicator_MACD():
        def main(self, df: pd.DataFrame, macdPeriod1: int, macdPeriod2: int, signalPeriod: int):
            df[f'MACD_{macdPeriod1},{macdPeriod2}'] = Indicator_EMA.main(self, df, macdPeriod1)[
                f'EMA_{macdPeriod1}'] - Indicator_EMA.main(self, df, macdPeriod2)[f'EMA_{macdPeriod2}']

            signal = []
            k = 2/(signalPeriod+1)

            for i in range(0, signalPeriod):
                signal.append(0)

            signal.append((df[f'MACD_{macdPeriod1},{macdPeriod2}'][(
                len(df)-signalPeriod):len(df)]).mean())

            for i in range(signalPeriod, len(df)-1):
                signal.append(
                    ((df[f'MACD_{macdPeriod1},{macdPeriod2}'][(len(df)-1)-i])*k) + ((signal[-1])*(1-k)))

            signal.reverse()
            df[f'MACDsignal_{signalPeriod}'] = signal
            return df

        def eval(self, df: pd.DataFrame):
            pass

    # RSI Indecator

    class Indicator_RSI():
        def main(self, df: pd.DataFrame, period: int):
            RSI = []
            for i in range(0, period):
                RSI.append(0)

            sumGain = 0
            sumLoss = 0
            for i in range(0, period):
                if df['Close'][(len(df)-2)-i] > df["Close"][(len(df)-1)-i]:
                    sumGain += (df['Close'][(len(df)-2)-i] -
                                df["Close"][(len(df)-1)-i])
                elif df['Close'][(len(df)-2)-i] == df["Close"][(len(df)-1)-i]:
                    pass
                else:
                    sumLoss += (df["Close"][(len(df)-1)-i] -
                                df['Close'][(len(df)-2)-i])
            averageGain = sumGain/period
            averageLoss = sumLoss/period
            RS = averageGain/averageLoss
            RSI.append(100-(100/(1+RS)))

            for i in range(period, len(df)-1):
                if df['Close'][(len(df)-2)-i] > df["Close"][(len(df)-1)-i]:
                    averageGain = ((averageGain*(period-1)) + (
                        df['Close'][(len(df)-2)-i] - df["Close"][(len(df)-1)-i]))/period
                elif df['Close'][(len(df)-2)-i] == df["Close"][(len(df)-1)-i]:
                    pass
                else:
                    averageLoss = ((averageLoss*(period-1)) + (
                        df["Close"][(len(df)-1)-i] - df['Close'][(len(df)-2)-i]))/period
                RS = averageGain/averageLoss
                RSI.append(100-(100/(1+RS)))

            RSI.reverse()
            df[f'RSI_{period}'] = RSI
            return df

        def eval(self, df: pd.DataFrame, period: int, trend: float):
            pass

    # On-Balance Volume(OBV):
    class Indicator_OBV():
        def main(self, df: pd.DataFrame):
            OBV = []
            OBV.append(df['Volume'][len(df)-1])
            for i in range(1, len(df)):
                if df['Close'][(len(df)-1)-i] > df["Close"][(len(df))-i]:
                    OBV.append(OBV[-1]+(df['Volume'][len(df)-i]))
                elif df['Close'][(len(df)-1)-i] == df["Close"][(len(df))-i]:
                    OBV.append(OBV[-1])
                else:
                    OBV.append(OBV[-1]-(df['Volume'][len(df)-i]))
            OBV.reverse()
            df['OBV'] = OBV
        
        def eval(self, df: pd.DataFram):
            pass
    
    # Accumulation/Distribution(A_D):
    class Indicator_A_D():
        def main(self, df: pd.DataFrame):
            A_D = []

            MFM = (((df['Close'][len(df)-1] - df['Low'][len(df)-1])-(df['High'][len(df)-1] -
                df['Close'][len(df)-1]))/(df['High'][len(df)-1] - df['Low'][len(df)-1]))
            MFV = MFM * df['Volume'][len(df)-1]
            A_D.append(MFV)

            for i in range(1, len(df)):
                MFM = (((df['Close'][(len(df)-1)-i] - df['Low'][(len(df)-1)-i])-(df['High'][(len(df)-1)-i] -
                    df['Close'][(len(df)-1)-i]))/(df['High'][(len(df)-1)-i] - df['Low'][(len(df)-1)-i]))
                MFV = MFM * df['Volume'][(len(df)-1)-i]
                A_D.append(A_D[-1]+MFV)

            A_D.reverse()
            df["A/D"] = A_D
            return df
        def eval(self, df: pd.DataFrame):
            pass

    # Average Directional Index(ADX):

    class Indicator_ADX():
        def main(self, df: pd.DataFrame, period: int):
            TR = [0]
            DM_positive = [0]
            DM_negative = [0]

            for i in range(1, len(df)):
                TR.append(df['High'][(len(df)-1)-i] -
                        df['Low'][(len(df)-1)-i])
                if (df['High'][(len(df)-1)-i] - df['High'][(len(df))-i]) > (df['Low'][(len(df))-i] - df['Low'][(len(df)-1)-i]):
                    DM_positive.append(
                        df['High'][(len(df)-1)-i] - df['High'][(len(df))-i])
                    DM_negative.append(0)
                elif (df['High'][(len(df)-1)-i] - df['High'][(len(df))-i]) < (df['Low'][(len(df))-i] - df['Low'][(len(df)-1)-i]):
                    DM_positive.append(0)
                    DM_negative.append(
                        df['Low'][(len(df))-i] - df['Low'][(len(df)-1)-i])
                else:
                    DM_positive.append(0)
                    DM_negative.append(0)

            STR = []
            SDM_positive = []
            SDM_negative = []
            for i in range(0, period):
                STR.append(0)
                SDM_positive.append(0)
                SDM_negative.append(0)

            STR.append(sum(TR[1:(period+1)]))
            SDM_positive.append(sum(DM_positive[1:(period+1)]))
            SDM_negative.append(sum(DM_negative[1:(period+1)]))

            for i in range(period+1, len(df)):
                SDM_positive.append(
                    SDM_positive[-1]-(SDM_positive[-1]/period) + DM_positive[i])
                SDM_negative.append(
                    SDM_negative[-1]-(SDM_negative[-1]/period) + DM_negative[i])
                STR.append(STR[-1]-(STR[-1]/period) + TR[i])

            DI_positive = []
            DI_negative = []
            DX = []

            for i in range(0, period):
                DI_positive.append(0)
                DI_negative.append(0)
                DX.append(0)

            DI_positive = DI_positive + \
                [(SDM_positive[i]/STR[i])*100 for i in range(period, len(df))]
            DI_negative = DI_negative + \
                [(SDM_negative[i]/STR[i])*100 for i in range(period, len(df))]

            DX = DX + [abs((DI_positive[i]-DI_negative[i])/(DI_positive[i] +
                        DI_negative[i]))*100 for i in range(period, len(df))]

            ADX = []
            for i in range(0, (period*2)-1):
                ADX.append(50)

            ADX.append(statistics.mean(DX[period:period*2]))
            for i in range(period*2, len(df)):
                ADX.append(((ADX[-1]*(period-1))+DX[i])/period)

            ADX.reverse()
            DI_positive.reverse()
            DI_negative.reverse()

            df[f'ADX_{period}'] = ADX
            df[f'+DI_{period}'] = DI_positive
            df[f'-DI_{period}'] = DI_negative

        def eval(self, df: pd.DataFrame):
            pass

    # Aroon Indicator/Oscillator:
    class Indicator_Aroon():
        def main(self, df: pd.DataFrame, period: int):
            aroonUp = []
            aroonDown = []
            aroonOsc = []

            for i in range(0, period):
                aroonUp.append(50)
                aroonDown.append(50)

            for i in range(period, len(df)):
                High_max = max(df['High'][len(df)-i:(len(df)-i)+period])
                index_of_high_max = df[df['High'] == High_max].index.values
                aroonUp.append(((period-(index_of_high_max[0]-(len(df)-i)))/period)*100)

                Low_min = min(df['Low'][len(df)-i:(len(df)-i)+period])
                index_of_low_min = df[df['Low'] == Low_min].index.values
                aroonDown.append(
                    ((period-(index_of_low_min[0]-(len(df)-i)))/period)*100)

            aroonOsc = [aroonUp[i]-aroonDown[i] for i in range(0, len(df))]

            aroonUp.reverse()
            aroonDown.reverse()
            aroonOsc.reverse()

            df[f'AroonUP_{period}'] = aroonUp
            df[f'AroonDown_{period}'] = aroonDown
            df[f'AroonOsc_{period}'] = aroonOsc
        
        def eval(self, df: pd.DataFrame):
            pass

    # Stochastic Oscillator(%k and %D)
    
    class Indicator_Stoc():
        def main(self, df: pd.DataFrame, period: int, subPeriod = 3):
            Stoc = []
            
            for i in range(0, period):
                Stoc.append(50)
            for i in range(period, len(df)):
                High_max = max(df['High'][len(df)-i:(len(df)-i)+period])
                Low_min = min(df['Low'][len(df)-i:(len(df)-i)+period])
                Stoc.append(
                    ((df['Close'][len(df)-i]-Low_min)/(High_max-Low_min))*100)

            Stoc.reverse()

            Stoc_D = []

            for i in range(0, len(df)-subPeriod):
                Stoc_D.append(statistics.mean(Stoc[i:i+subPeriod]))
            for n in range(0, subPeriod):
                Stoc_D.append(0)

            df[f'Stochastic%K_{period}'] = Stoc
            df[f'Stochastic%D_{subPeriod}'] = Stoc_D
            
            return df

        def eval(self, df:pd.DataFrame):
            pass

    # Contribution:
    class Indicator_Contribution():
        def main(self, df: pd.DataFrame):
            contribution_List = []

            for i in range(0, len(df)):
                contribution_List.append(df['Transaction'][i])

            contribution_List = [math.log(contribution_List[x], 10)
                                for x in range(0, len(contribution_List))]
            contribution_List = [(df['Volume'][i])/(contribution_List[i])
                                for i in range(0, len(contribution_List))]
            df['Contribution'] = contribution_List

            return df
        
        def eval(self, df: pd.DataFrame):
            pass




In [ ]:


ind = Indicator_SMA()
ind.main
print(df.head)

In [63]:
# Candle_patterns

if Candle_patterns==True:
    suc = 0
    fai = 0
    for i in range(2, window-1):
        if (df['High'][i] - df['Open'][i]) / 2 > (df['Open'][i] - df['Close'][i]):
            if (df['Open'][i] - df['Close'][i] > df['Close'][i] - df['Low'][i]):
                if df['Close'][i-2] > df['Open'][i-2]:
                    if df['Close'][i-1] > df['Open'][i-1]:
                        if df['Close'][i+1] < df['Open'][i+1]:
                            suc += 1
                        else:
                            fai += 1


In [64]:
# Extermum points

extermum_df = pd.DataFrame(columns=window_df.columns)
subWindow = 200


for i in range(1, subWindow-1):
    if (window_df['Close'][i] > window_df['Close'][i-1] and window_df['Close'][i] > window_df['Close'][i+1]) or (window_df['Close'][i] < window_df['Close'][i-1] and window_df['Close'][i] < window_df['Close'][i+1]):
        extermum_df.loc[i] = window_df.loc[i]



In [65]:
# Valuable extermum point (VE point)

VE_df = pd.DataFrame(columns=window_df.columns)
subWindow = 200

volume_Mode = statistics.mode((window_df['Volume'][0:subWindow]).astype(int))

for i in range(1, subWindow-1):
    if window_df['Volume'][i] > volume_Mode*5 and window_df['Volume'][i] > window_df['Volume'][i-1]:
        a = i
        
        while True:
            if (window_df['Close'][a] > window_df['Close'][a-1] and window_df['Close'][a] > window_df['Close'][a+1]) or (window_df['Close'][a] < window_df['Close'][a-1] and window_df['Close'][a] < window_df['Close'][a+1]):
                VE_df.loc[a] = window_df.loc[a]
                break
            else:
                a -= 1
                if a==0:
                    break
print(len(VE_df))



10


In [66]:
# Trend
if len(VE_df) >= 2:
    linearFunction_df = pd.DataFrame(VE_df).sort_values(
        by=['Close'], ascending=False, ignore_index=True)


    def linearFunction(point_1=None, point_2=None, x_points=None, y_points=None, LFm=None, LFb=None, x=None, y=None):

        if x_points != None and y_points != None:
            LFm = (y_points[0]-y_points[1])/(x_points[0]-x_points[1])
            LFb = y_points[0] - LFm*x_points[0]
            if y == None and x != None:
                y = LFm*x + LFb
                return LFm, LFb, y
            if x == None and y != None:
                x = (y - LFb)/LFm
                return LFm, LFb, x
            if y == None and x == None:
                return LFm, LFb

        if x != None and y != None and LFm != None:
            LFb = y - LFm*x
            return LFb

        if point_1 != None and point_2 != None:
            LFm = (point_2[0]-point_1[0])/(point_2[1]-point_1[1])
            LFb = point_1[0] - LFm*point_1[1]
            return LFm, LFb


    # maximum and minimum aproach:
    print('maximum and minimum aproach:')

    maxPoints_df = pd.DataFrame(linearFunction_df[['Close', 'Time']][0: 2]).sort_values(
        by=['Time'], ascending=False, ignore_index=True).astype(float)

    minPoints_df = pd.DataFrame(linearFunction_df[['Close', 'Time']][len(linearFunction_df['Close'])-2: len(linearFunction_df['Close'])]).sort_values(
        by=['Time'], ascending=False, ignore_index=True).astype(float)


    max_P = [maxPoints_df['Close'][0], maxPoints_df['Close'][1]]
    min_P = [minPoints_df['Close'][0], minPoints_df['Close'][1]]
    max_TP = [maxPoints_df['Time'][0], maxPoints_df['Time'][1]]
    min_TP = [minPoints_df['Time'][0], minPoints_df['Time'][1]]

    line_max = list(linearFunction(x_points=max_TP, y_points=max_P))
    line_min = list(linearFunction(x_points=min_TP, y_points=min_P))
    if line_max[0] > 0 and line_min[0] > 0:
        if line_max[0] > line_max[0]:
            line_min[0] = line_min[0]
            line_min[1] = linearFunction(x=min_TP[0], y=min_P[0], LFm=line_min[0])
            trendType = 'ascending'
        if line_max[0] < line_min[0]:
            line_max[0] = line_min[0]
            line_max[1] = linearFunction(x=max_TP[1], y=max_P[1], LFm=line_max[0])
            trendType = 'ascending'
    elif line_max[0] < 0 and line_min[0] < 0:
        if line_max[0] > line_min[0]:
            line_max[0] = line_min[0]
            line_max[1] = linearFunction(x=max_TP[1], y=max_P[1], LFm=line_max[0])
            trendType = 'descending'
        if line_max[0] < line_min[0]:
            line_min[0] = line_max[0]
            line_min[1] = linearFunction(x=min_TP[0], y=min_P[0], LFm=line_min[0])
            trendType = 'descending'
    elif line_max[0] < 0 and line_min[0] > 0:
        trendType = 'convergent'
    elif line_max[0] > 0 and line_min[0] < 0:
        trendType = 'divergent'
    elif line_max[0] == line_min[0]:
        trendType = 'ERROR'
    else:
        trendType = 'nan'

    print(line_max)
    print(line_min)
    print(trendType)

    # line strength aproach:
    print('\n\nline strength aproach:')

    proximity = subWindow/5

    lines = []
    n = 0
    linearFunction_df = linearFunction_df.sort_values(
        by=['Time'], ascending=False, ignore_index=True)

    for data1 in range(n, len(linearFunction_df['Close'])):
        a = data1
        data1 = [linearFunction_df.loc[data1]['Close'],
                float(linearFunction_df.loc[data1]['Time'])]

        for data2 in range(a+1, len(linearFunction_df['Close'])):
            data2 = [linearFunction_df.loc[data2]['Close'],
                    float(linearFunction_df.loc[data2]['Time'])]
            if (data1[1]-data2[1])/type_int > proximity:
                lines.append(linearFunction(point_1=data2, point_2=data1))

    print(len(lines))


maximum and minimum aproach:
[-0.01101360544217697, 18195923.12448996]
[-0.01101360544217697, 18194779.726530775]
descending


line strength aproach:
16
